### Requires latest toytree

In [2]:
import toytree
toytree.__version__

'1.0.9'

### Code modified from ivy.phylo

In [3]:

def PIC(tree, feature):
    """
    Infer ancestral states and calculate phylogenetic independent
    contrasts at nodes for a selected feature (trait). 
    
    Modified from IVY interactive (https://github.com/rhr/ivy/)
    
    Parameters
    ----------
    """
    # make a copy to avoid modifying tree
    ntre = tree.copy()
    
    # get current node features at the tips
    fdict = ntre.get_feature_dict(key_attr="name", values_attr=feature)
    data = {i: j for (i, j) in fdict.items() if i in ntre.get_tip_labels()}

    # apply dynamic function from ivy to return dict results
    results = dynamicPIC(ntre.treenode, data, results={})
    
    # set mean, variance, contrast, contrast-var as new node attrs
    restype = ["mean", "var", "contrast", "contrast-var"]
    
    # add ancestral values
    for idx, res in enumerate(restype):        
        ntre = ntre.set_node_values(
            "{}-{}".format(feature, res), 
            values={i.name: j[idx] for i,j in results.items()},
            default=0,
        )
    
    # add tip values back in for mean
    ntre = ntre.set_node_values("{}-mean".format(feature), data)           
        
    # return modified tree
    return ntre


def dynamicPIC(node, data, results):
    """
    Phylogenetic independent contrasts. Recursively calculate 
    independent contrasts of a bifurcating node given a dictionary
    of trait values.
    
    Modified from IVY interactive (https://github.com/rhr/ivy/)

    Args:
        node (Node): A node object
        data (dict): Mapping of leaf names to character values
    Returns:
        dict: Mapping of internal nodes to tuples containing ancestral
              state, its variance (error), the contrast, and the
              contrasts's variance.
    TODO: modify to accommodate polytomies.
    """    
    X = []; v = []
    
    # recurively does children until X and v are full
    for child in node.children:
        
        # child has children, do those first
        if child.children:
            
            # update results dict with children values
            dynamicPIC(child, data, results)
            child_results = results[child]
            
            # store childrens values
            X.append(child_results[0])
            v.append(child_results[1])
            
        # no child of child, so just do child
        else:
            X.append(data[child.name])
            v.append(child.dist)

    # Xi - Xj is the contrast value
    Xi, Xj = X  
    
    # vi + vj is the contrast variance
    vi, vj = v

    # Xk is the reconstructed state at the node
    Xk = ((1.0/vi)*Xi + (1/vj)*Xj) / (1.0/vi + 1.0/vj)

    # vk is the variance
    vk = node.dist + (vi*vj)/(vi+vj)
    
    # store in dictionary and 
    results[node] = (Xk, vk, Xi-Xj, vi+vj)
    return results


### Example: 

In [4]:
import toyplot
colormap = toyplot.color.brewer.map("BlueRed", reverse=True)
colormap

In [5]:
tree = toytree.rtree.imbtree(5, 1e6)
tree = tree.set_node_values(
    "g", 
    values={i: 5 for i in (2, 3, 4)},
    default=1,
)
tree.draw(
    ts='p', 
    node_labels=tree.get_node_values("g", 1, 1),
    node_colors=[colormap.colors(i, 0, 5) for i in tree.get_node_values('g', 1, 1)]
);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t84fe8b348d39445e82fbc5ab7e36161a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1 r0 r2 r3 r4 1 1 5 5 5 1 1 1 1 0 500000 1000000

In [6]:
# apply reconstruction
ntree = PIC(tree, "g")

# new values are stored as -mean, -var, -contrasts, ...
evals = ntree.get_edge_values("g-mean")

# draw new tree
ntree.draw(
    ts='p', 
    node_labels=ntree.get_node_values("g-mean", 1, 1),
    node_colors=[colormap.colors(i, 0, 5) for i in ntree.get_node_values('g-mean', 1, 1)]
);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="te02b97e8c8654b69bc138c9f82dc33ee" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1 r0 r2 r3 r4 1.0 1.0 5.0 5.0 5.0 1.0 2.7142857142857135 3.5882352941176463 4.08133971291866 0 500000 1000000